<a href="https://colab.research.google.com/github/stepanjaburek/quantum_social_science_lr/blob/main/permutated_tests_colab_HCPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Social Science: PCA and Cluster Analysis of the Literature

#Setup

In [ ]:
!pip install scikit-learn-extra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from IPython.display import display

In [216]:
from google.colab import drive
drive.mount('/content/drive') # this asks for your google account
# load data from our private google folder shared among us
data = pd.read_excel("/content/drive/MyDrive/QSS_Colab/keyword_data.xlsx")
metadata = pd.read_excel("/content/drive/MyDrive/QSS_Colab/metadata.xlsx")
data1 = pd.read_excel("/content/drive/MyDrive/QSS_Colab/1_dataset_with_all_21_variables.xlsx")
data2 = pd.read_excel("/content/drive/MyDrive/QSS_Colab/2_dataset_with_just_6_variables.xlsx")
data3 = pd.read_excel("/content/drive/MyDrive/QSS_Colab/3_permutation_in_columns.xlsx")
data4 = pd.read_excel("/content/drive/MyDrive/QSS_Colab/4_permutation_in_columns_6_stays.xlsx")
data5 = pd.read_excel("/content/drive/MyDrive/QSS_Colab/5_permutation_in_columns_15_stays.xlsx")
data6 = pd.read_excel("/content/drive/MyDrive/QSS_Colab/6_permutation_in_rows.xlsx")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data cleaning

In [217]:
# Rename "Broekaert_2018" in metadata
metadata.loc[252, 'file_name'] = "Broekaert_2018_The Tacit 'Quantum' of Meeting the Aesthetic Sign; Contextualize, Entangle,2.pdf"
data1.loc[252, 'filename'] = "Broekaert_2018_The Tacit 'Quantum' of Meeting the Aesthetic Sign; Contextualize, Entangle,2.pdf"
# Delete duplicit paper "Yukalov et al. - 2018 - Information processing by networks of quantum deci.pdf"
data1 = data1.drop(1170)
# Delete paper "Yilmaz - 2017 - Quantum cognition models of ethical decision-makin.pdf" that is only present in data
data1 = data1.drop(1166)
# Delete paper "Park_2016_Decision-making &amp quantum mechanical models of cognitive processing.pdf" that is only present in metadata
metadata = metadata.drop(892)

In [101]:
data2.loc[252, 'filename'] = "Broekaert_2018_The Tacit 'Quantum' of Meeting the Aesthetic Sign; Contextualize, Entangle,2.pdf"
# Delete duplicit paper "Yukalov et al. - 2018 - Information processing by networks of quantum deci.pdf"
data2 = data2.drop(1170)
# Delete paper "Yilmaz - 2017 - Quantum cognition models of ethical decision-makin.pdf" that is only present in data
data2 = data2.drop(1166)

In [147]:
data3.loc[252, 'filename'] = "Broekaert_2018_The Tacit 'Quantum' of Meeting the Aesthetic Sign; Contextualize, Entangle,2.pdf"
# Delete duplicit paper "Yukalov et al. - 2018 - Information processing by networks of quantum deci.pdf"
data3 = data3.drop(1170)
# Delete paper "Yilmaz - 2017 - Quantum cognition models of ethical decision-makin.pdf" that is only present in data
data3 = data3.drop(1166)

In [117]:
data4.loc[252, 'filename'] = "Broekaert_2018_The Tacit 'Quantum' of Meeting the Aesthetic Sign; Contextualize, Entangle,2.pdf"
# Delete duplicit paper "Yukalov et al. - 2018 - Information processing by networks of quantum deci.pdf"
data4 = data4.drop(1170)
# Delete paper "Yilmaz - 2017 - Quantum cognition models of ethical decision-makin.pdf" that is only present in data
data4 = data4.drop(1166)

In [159]:
data5.loc[252, 'filename'] = "Broekaert_2018_The Tacit 'Quantum' of Meeting the Aesthetic Sign; Contextualize, Entangle,2.pdf"
# Delete duplicit paper "Yukalov et al. - 2018 - Information processing by networks of quantum deci.pdf"
data5 = data5.drop(1170)
# Delete paper "Yilmaz - 2017 - Quantum cognition models of ethical decision-makin.pdf" that is only present in data
data5 = data5.drop(1166)

In [167]:
data6.loc[252, 'filename'] = "Broekaert_2018_The Tacit 'Quantum' of Meeting the Aesthetic Sign; Contextualize, Entangle,2.pdf"
# Delete duplicit paper "Yukalov et al. - 2018 - Information processing by networks of quantum deci.pdf"
data6 = data6.drop(1170)
# Delete paper "Yilmaz - 2017 - Quantum cognition models of ethical decision-makin.pdf" that is only present in data
data6 = data6.drop(1166)

In [218]:
df=data1

In [219]:
df=data1
metadata.insert(0, 'id', range(1, len(metadata) + 1))
df.insert(0, 'id', range(1, len(df) + 1))

In [ ]:
df

# Keywords

In [224]:
df = df[df.iloc[:, 2:23].sum(axis=1) > 0]
df = df.reset_index(drop=True)

In [ ]:
[(i, list(df.keys())[i]) for i in range(len(df.keys()))]

# Scaling in the rows

In [ ]:
# create features for the pca
features = df.drop(['filename', 'id'], axis=1)
feature_names = features.columns

features = features.div(features.sum(axis=1), axis=0) # row normalization as Michael suggested, dividing the values by the sum of their row (paper)
#features = StandardScaler().fit_transform(features.T).T # Z-score standardization on transposed data to work in rows

features = pd.DataFrame(features, columns=feature_names)
features

# Data analysis

#PCA

In [ ]:
pca = PCA()
pca_result = pca.fit_transform(features)
print("Explained variance ratio:", pca.explained_variance_ratio_[:10])

# Scree plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1),
         pca.explained_variance_ratio_, 'bo-')
plt.title('Scree Plot of Principal Components')
plt.xlabel('Principal Components')
plt.ylabel('Percentage of explained variances')
plt.show()

# Feature importance in components
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(len(pca.components_))],
    index=features.columns
)

# Top 5 features per component
top_loadings = pd.DataFrame()
for pc in loadings.columns:
    top_5 = pd.DataFrame({
        'feature': loadings.index,
        'PC': pc,
        'loading': loadings[pc]
    })

    top_5 = top_5.reindex(top_5['loading'].abs().sort_values(ascending=False).index)
    top_5 = top_5.head(5)
    top_loadings = pd.concat([top_loadings, top_5])

print(top_loadings)

# PC1 and 2
pc12_loadings = top_loadings[top_loadings['PC'].isin(['PC1', 'PC2', 'PC3', 'PC4'])]
pc12_loadings = pc12_loadings.sort_values(['PC', 'loading'],
                                         ascending=[True, False])
print(pc12_loadings)

# PCA biplot
plt.figure(figsize=(12, 8))
plt.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.5)

for i, feature in enumerate(features.columns):
    plt.arrow(0, 0,
              pca.components_[0, i]*max(abs(pca_result[:, 0])),
              pca.components_[1, i]*max(abs(pca_result[:, 1])),
              color='r', alpha=0.5)
    plt.text(pca.components_[0, i]*max(abs(pca_result[:, 0]))*1.15,
             pca.components_[1, i]*max(abs(pca_result[:, 1]))*1.15,
             feature)

plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('PCA of Paper Concepts')
plt.show()
display(pc12_loadings)

How many PCs do we want? Literature says probably a sum between 80-90% of explained variance

In [ ]:
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
# Find number of components needed for 80% variance (we can change this)
n_components_80 = np.argmax(cumulative_variance_ratio >= 0.8) + 1

print(f"Number of components needed to explain 80% of variance: {n_components_80}")

# Visualization
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance_ratio) + 1),
         cumulative_variance_ratio,
         'bo-')
plt.axhline(y=0.8, color='r', linestyle='--', label='80% Threshold')
plt.axvline(x=n_components_80, color='g', linestyle='--',
            label=f'Components needed: {n_components_80}')
plt.title('Cumulative Explained Variance Ratio')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.legend()
plt.grid(True)
plt.show()

print("\nCumulative explained variance for first 10 components:")
for i in range(10):
    print(f"Components 1-{i+1}: {cumulative_variance_ratio[i]:.3f}")

# Hierarchical Clustering on Principal Components (HCPC)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

# 1. Define how many PCs to use
n_components = 7 # I think we should use between 7 and 9 based on the PCA
selected_components = pca_result[:, :n_components]

# 2. Hierarchical clustering uisng Ward
linked = linkage(selected_components, method='ward') # the methods can be different. Ward method is the one we use in Ipsos and seemingly the typical choice in the literature

# Plot dendrogram
plt.figure(figsize=(12, 8))
dendrogram(linked, truncate_mode='lastp', p=12)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.show()

# 3. Get initial clusters from hierarchical clustering # optional step though, we dont really need them
n_clusters = 4
hc_labels = fcluster(linked, n_clusters, criterion='maxclust')

# 4. K-means to finalize things
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(selected_components)
final_labels = kmeans.labels_


# Visualize final clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1],
                     c=final_labels, cmap='viridis')
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
           c='red', marker='x', s=200, linewidths=3, label='Centroids')
plt.colorbar(scatter)
plt.title('HCPC Final Clusters')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.legend()
plt.show()

K-Medoids as an alternative ending

In [ ]:
kmedoids = KMedoids(n_clusters=n_clusters,
                    random_state=42,
                    metric='euclidean')
kmedoids.fit(selected_components)
final_labels = kmedoids.labels_

# Visualize final clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1],
                     c=final_labels, cmap='viridis')

# Get medoid locations in PCA space
medoid_indices = kmedoids.medoid_indices_
medoid_locations = pca_result[medoid_indices]

# Plot medoids
plt.scatter(medoid_locations[:, 0], medoid_locations[:, 1],
           c='red', marker='x', s=200, linewidths=3, label='Medoids')

plt.colorbar(scatter)
plt.title('HCPC Final Clusters (K-medoids)')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.legend()
plt.show()

# Get central papers from the HCPC

Kmeans

In [202]:
centroids_list = []
for i in range(n_clusters):
    distances = np.sum((selected_components - kmeans.cluster_centers_[i])**2, axis=1)
    top_indices = np.argsort(distances)[:10] # N of central papers
    pca_coords = pca_result[top_indices]

    feature_importance = np.dot(pca_coords[:, :7], pca.components_[:7, :])
    feature_importance_df = pd.DataFrame(
        feature_importance,
        columns=features.columns
    )

    # Get top 5 most important features for each paper
    top_features = pd.DataFrame({
        'paper_id': df['id'].iloc[top_indices].values,
        'top_features': [
            ', '.join(feature_importance_df.iloc[j].nlargest(5).index.tolist())
            for j in range(len(feature_importance_df))
        ]
    })
        # Get PCA coordinates for the top papers, here we work with the 7 PCs, we can change it
    cluster_papers = pd.DataFrame({
        'cluster': [i + 1] * 10,
        'rank': range(1, 11),
        'centroid_id': df['id'].iloc[top_indices].values,
        'distance': distances[top_indices],
        'PC1': pca_coords[:, 0],
        'PC2': pca_coords[:, 1],
        'PC3': pca_coords[:, 2],
        'PC4': pca_coords[:, 3],
        'PC5': pca_coords[:, 4],
        'PC6': pca_coords[:, 5],
        'PC7': pca_coords[:, 6],
        'key_features': top_features['top_features']
    })
    centroids_list.append(cluster_papers)

centroids = pd.concat(centroids_list, ignore_index=True)
centerpapers = pd.merge(centroids, metadata, left_on='centroid_id', right_on='id', how='left')
centerpapers.to_csv('means_centroids_no_operator.csv', index=False)

In [ ]:
centerpapers

KMedoids

In [98]:
centroids_list = []
for i in range(n_clusters):
    medoid_point = selected_components[kmedoids.medoid_indices_[i]]
    distances = np.sum((selected_components - medoid_point)**2, axis=1)
    top_indices = np.argsort(distances)[:10]  # N of central papers
    pca_coords = pca_result[top_indices]

    feature_importance = np.dot(pca_coords[:, :7], pca.components_[:7, :])
    feature_importance_df = pd.DataFrame(
        feature_importance,
        columns=features.columns
    )

    # Get top 5 most important features for each paper
    top_features = pd.DataFrame({
        'paper_id': df['id'].iloc[top_indices].values,
        'top_features': [
            ', '.join(feature_importance_df.iloc[j].nlargest(5).index.tolist())
            for j in range(len(feature_importance_df))
        ]
    })
     # Get PCA coordinates for the top papers, here we work with the 8 PCs, we can change it
    cluster_papers = pd.DataFrame({
        'cluster': [i + 1] * 10,
        'rank': range(1, 11),
        'centroid_id': df['id'].iloc[top_indices].values,
        'distance': distances[top_indices],
        'PC1': pca_coords[:, 0],
        'PC2': pca_coords[:, 1],
        'PC3': pca_coords[:, 2],
        'PC4': pca_coords[:, 3],
        'PC5': pca_coords[:, 4],
        'PC6': pca_coords[:, 5],
        'PC7': pca_coords[:, 6],
        'key_features': top_features['top_features']
    })
    centroids_list.append(cluster_papers)

centroids = pd.concat(centroids_list, ignore_index=True)
centerpapers = pd.merge(centroids, metadata, left_on='centroid_id', right_on='id', how='left')
centerpapers.to_csv('medoids_centroids_no_operator.csv', index=False)

In [99]:
clustered_df = df.copy()
clustered_df['cluster'] = final_labels + 1

clustered_df.to_csv('papers_with_clusters.csv', index=False)

# K-Means vs K-Medoids

In [ ]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, normalized_mutual_info_score
kmeans_labels = kmeans.labels_
kmedoids_labels = kmedoids.labels_

# similarity metrics
ari = adjusted_rand_score(kmeans_labels, kmedoids_labels)
ami = adjusted_mutual_info_score(kmeans_labels, kmedoids_labels)
nmi = normalized_mutual_info_score(kmeans_labels, kmedoids_labels)

print(f"Adjusted Rand Index: {ari:.3f}")
print(f"Adjusted Mutual Information: {ami:.3f}")
print(f"Normalized Mutual Information: {nmi:.3f}")

confusion_matrix = pd.crosstab(
    kmeans_labels,
    kmedoids_labels,
    margins=True,
    normalize='index'
)
print("\nConfusion Matrix (normalized by row):")
print(confusion_matrix)

# Visual comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# K-means plot
scatter1 = ax1.scatter(pca_result[:, 0], pca_result[:, 1], c=kmeans_labels, cmap='viridis')
ax1.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
           c='red', marker='x', s=200, linewidths=3, label='Centroids')
ax1.set_title('K-means Clusters')
ax1.set_xlabel('First Principal Component')
ax1.set_ylabel('Second Principal Component')
ax1.legend()

# K-medoids plot
scatter2 = ax2.scatter(pca_result[:, 0], pca_result[:, 1], c=kmedoids_labels, cmap='viridis')
ax2.scatter(medoid_locations[:, 0], medoid_locations[:, 1],
           c='red', marker='x', s=200, linewidths=3, label='Medoids')
ax2.set_title('K-medoids Clusters')
ax2.set_xlabel('First Principal Component')
ax2.set_ylabel('Second Principal Component')
ax2.legend()

plt.tight_layout()
plt.show()

# Somewhat good overlap of K-Means and K-Medoids with 7 PCs and 4 clusters. ARI=0.577; C1 (95.2%), C2 (94.4%), C3 (98.8%), C4 (38%) and (61.4%)

# Very good overlap of K-Means and K-Medoids with 9 PCs and 4 clusters. ARI=0.77; C1 (80.4% overlap), C2(88.5% overlap), C3 (95.6% overlap), C4 (100% overlap)

# Random Forest for a ML approach to "predicting" the clusters we got from the PCA a Cluster Analysis with the just the original (normalized) keywords.



Gives us the same picture as the PCA = Operator, Uncertainty, and Entanglement predict one cluster each, the fourth is predicted by a mix of words. Not sure we get any useful info here. **But the model fits extremely well all clusters from the original data, this is good robustness for the pca and clsuter approach we have.**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = features
y = final_labels + 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf.fit(X_train, y_train)


feature_importance = pd.DataFrame({
    'feature': features.columns,
    'importance': rf.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

# 20 most important features
print("Top 20 most important features:")
print(feature_importance.head(20))

# Model performance
y_pred = rf.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# viz
plt.figure(figsize=(12, 6))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
plt.title('Top 20 Most Important Features for Cluster Prediction')
plt.xlabel('Feature Importance')
plt.tight_layout()
plt.show()

In [ ]:
for cluster_num in range(1, 5):
    print(f"\n=== ANALYSIS FOR CLUSTER {cluster_num} ===")

    binary_labels = (final_labels + 1 == cluster_num).astype(int)
    X_train, X_test, y_train, y_test = train_test_split(
        features, binary_labels, test_size=0.2, random_state=42
    )
    rf = RandomForestClassifier(n_estimators=1000, random_state=42)
    rf.fit(X_train, y_train)


    feature_importance = pd.DataFrame({
        'feature': features.columns,
        'importance': rf.feature_importances_
    })
    feature_importance = feature_importance.sort_values('importance', ascending=False)

    ################################################

    # top 10 features for this cluster
    print(f"\nTop 10 most important features for Cluster {cluster_num}:")
    print(feature_importance.head(10))

    # Model performance
    y_pred = rf.predict(X_test)
    print(f"\nClassification Report for Cluster {cluster_num}:")
    print(classification_report(y_test, y_pred))

    # viz
    plt.figure(figsize=(12, 6))
    sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
    plt.title(f'Top 20 Most Important Features for Cluster {cluster_num}')
    plt.xlabel('Feature Importance')
    plt.tight_layout()
    plt.show()